In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import imutils
import os
import tensorflow as tf
#from keras.preprocessing.image import load_img
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_path = "/content/images"
pts_path = "/content/points"
for filename in os.listdir(image_path):
    if filename=='.ipynb_checkpoints':
      continue
    else:
    face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
    image = cv2.imread(os.path.join(image_path, filename))
    data = im.fromarray(image)
    print(data.size)
# Convert into grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 1)
    imagetemp=image
# Draw rectangle around the faces
    for (x, y, w, h) in faces:
          cv2.rectangle(imagetemp, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output
    plt.imshow(imagetemp)


In [ ]:
def get_pts(file_pts):
    pts = file_pts.readlines()
    pts = pts[3:len(pts) - 1]rr
    points = np.zeros((len(pts), 2))
    for i in range(len(pts)):
        pts[i] = pts[i].replace('\n', '')
        x = pts[i].split(' ')
        points[i][0] = float(x[0])
        points[i][1] = float(x[1])

    angle = math.atan2(points[31, 1] - points[36, 1], points[31, 0] - points[36, 0]) * 180 / math.pi
    if angle > 90:
        angle = angle - 180
    if angle < -90:
        angle = angle + 180
    return points, angle

In [ ]:
def preprocessing():
    image_path = "/content/drive/MyDrive/FGNET/images"
    pts_path = "/content/drive/MyDrive/FGNET/points"

    for filename in os.listdir(image_path):
        image = cv2.imread(os.path.join(image_path, filename))
        name_pts = filename.split('.')[0].lower() + ".pts"

        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        file_pts = open(os.path.join(pts_path, name_pts))

        points, angle = get_pts(file_pts)

        rotated = imutils.rotate(image, angle)
        m_i = min(points[16][1], points[22][1])
        if filename != "033A30.JPG":
            croped = rotated[int(m_i - 10):int(points[7][1]), int(points[0][0] - 10):int(points[14][0] + 10)]
        else:
            croped = rotated[int(m_i - 10):int(points[7][1]), int(points[0][0]):int(points[14][0] + 5)]
        rot_crop_his = cv2.equalizeHist(croped)#It improves the contrast and brightness of an image in order to stretch out the intensity range
        try:
            cv2.imwrite(os.path.join("/content/drive/MyDrive/FGNET/images_new", filename), rot_crop_his)
        except:
            print(filename)

In [ ]:
def get_images_new():
    image_path = "/content/drive/MyDrive/FGNET/images_new"
    labels = np.zeros((1, 1002))
    i = 0
    images = []
    print(labels.shape)
    for filename in os.listdir(image_path):
        img = load_img(os.path.join(image_path, filename), target_size=(224, 224))
        img = img_to_array(img)
        images.append(img)
        labels[0][i] = int(filename.split('A')[0])
        i = i + 1
    return np.array(images), labels

In [ ]:
def feature_extraction():
    model = tf.keras.applications.VGG16(
        include_top=True,
        weights="imagenet",
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        classifier_activation="softmax",
    )

    images, labels = get_images_new()
    image_path = "/content/drive/MyDrive/FGNET/images"
    f = open('/content/drive/MyDrive/FGNET/flatten_feature.txt', 'w')
    f1 = open('/content/drive/MyDrive/FGNET/fc7_feature.txt', 'w')
    f2 = open('/content/drive/MyDrive/FGNET/fc6_feature.txt', 'w')
    l = len(model.layers)
    for i in range(len(images)):
        image = images[i]
        img = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

        keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 4).output)
        Flatten = keras_function([img])

        keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 3).output)
        FC7 = keras_function([img])

        keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 2).output)
        FC6 = keras_function([img])

        f.write(str(labels[0][i]) + '\n')
        f1.write(str(labels[0][i]) + '\n')
        f2.write(str(labels[0][i]) + '\n')
        np.savetxt(f, Flatten, fmt="%s")
        np.savetxt(f1, FC7, fmt="%s")
        np.savetxt(f2, FC6, fmt="%s")
    f.close()
    f1.close()
    f2.close()

In [ ]:
def ldp(photo):
  import cv2
  import numpy as np
  def assign_bit(picture, x, y, c1,c2,d):  #assign bit according to degree and neighbouring pixel
    #a and b are 1 if increasing and 0 if decreasing
    if (d==0):
      a = 0
      b =0
      try:
        if picture[c1][c2+1] >= picture[c1][c2]:
          a= 1
        if picture[x][y+1] >= picture[x][y]:
          b= 1
      except:
        pass
    if (d==45):
      a = 0
      b =0
      try:
        if picture[c1-1][c2+1] >= picture[c1][c2]:
          a= 1
        if picture[x-1][y+1] >= picture[x][y]:
          b= 1
      except:
        pass
    if (d==90):
      a = 0
      b =0
      try:
        if picture[c1-1][c2] >= picture[c1][c2]:
          a= 1
        if picture[x-1][y] >= picture[x][y]:
          b= 1
      except:
        pass
    if (d==135):
      a = 0
      b =0
      try:
        if picture[c1-1][c2-1] >= picture[c1][c2]:
          a= 1
        if picture[x-1][y-1] >= picture[x][y]:
          b= 1
      except:
        pass
    if (a==b): #if monotonically increasing or decreasing than 0
      return "0"
    else:      #if turning point
      return "1"
    return bit
  def local_der_val(picture, x, y):  #calculating local derivative pattern value of a pixel
    thirtytwo_bit_binary = []
    centre = picture[x][y]
    c1=x
    c2=y
    decimal_val = 0
    #starting from top left,assigning bit to pixels clockwise at 0 degree
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y-1,c1,c2,0))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y,c1,c2,0))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y + 1,c1,c2,0))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y + 1, c1,c2,0))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y + 1,c1,c2,0 ))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y,c1,c2,0))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y-1,c1,c2,0))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y-1,c1,c2,0))

    #starting from top left,assigning bit to pixels clockwise at 45 degree
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y-1,c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y,c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y + 1,c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y + 1, c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y + 1,c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y,c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y-1,c1,c2,45))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y-1,c1,c2,45))

    #starting from top left,assigning bit to pixels clockwise at 90 degree
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y-1,c1,c2,90))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y,c1,c2,90))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y + 1,c1,c2,90))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y + 1, c1,c2,90))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y + 1,c1,c2,90 ))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y,c1,c2,90))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y-1,c1,c2,90))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y-1,c1,c2,90))

    #starting from top left,assigning bit to pixels clockwise at 135 degree
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y-1,c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y,c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x-1, y + 1,c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y + 1, c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y + 1,c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y,c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x + 1, y-1,c1,c2,135))
    thirtytwo_bit_binary.append(assign_bit(picture, x, y-1,c1,c2,135))

    str1 = ""
    l=str1.join(thirtytwo_bit_binary)#32 bit binary number
    decimal_val=int(l,2)   #32 bit binary to decimal number
    return decimal_val
  m, n, _ = photo.shape
  gray_scale = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)  #converting image to grayscale
  ldp_photo = np.zeros((m, n),np.uint8)
  # converting image to ldp
  for i in range(0,m):
      for j in range(0,n):
          ldp_photo[i, j] = local_der_val(gray_scale, i, j)

  return ldp_photo

In [ ]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
#image_path = "/content/drive/MyDrive/FGNET/images"
images, labels = get_images_new()
f = open('/content/drive/MyDrive/FGNET/final_ldp.txt', 'w')
#for filename in os.listdir(image_path):
 #       img = cv2.imread(os.path.join(image_path,filename))
c=0.0
for i in range(len(images)):
        image = images[i]
        l1=ldp(image)
        l2=l1.reshape(1,-1)
        f.write(str(labels[0][i]) + '\n')
        np.savetxt(f, l2, fmt="%s")
        c=c+1

f.close()
print(c)

(1, 1002)
1002.0


In [ ]:
preprocessing()
feature_extraction()

553476096/553467096 [==============================] - 4s 0us/step
(1, 1002)


In [ ]:
function [Ax,Ay,Xs,Ys] = dcaFuse(X,Y,label)

[p,n] = size(X);
if size(Y,2) ~= n
    error('X and Y must have the same number of columns (samples).');
elseif length(label) ~= n
    error('The length of the label must be equal to the number of samples.');
elseif n == 1
    error('X and Y must have more than one column (samples)');
end
q = size(Y,1);

classes = unique(label);
c = numel(classes);
cellX = cell(1,c);
cellY = cell(1,c);
nSample = zeros(1,c);
for i = 1:c
    index = find(label==classes(i));
    nSample(i) = length(index);
    cellX{1,i} = X(:,index);
    cellY{1,i} = Y(:,index);
end

meanX = mean(X,2);  % Mean of all training data in X
meanY = mean(Y,2);  % Mean of all training data in Y

classMeanX = zeros(p,c);
classMeanY = zeros(q,c);
for i = 1:c
    classMeanX(:,i) = mean(cellX{1,i},2);   % Mean of each class in X
    classMeanY(:,i) = mean(cellY{1,i},2);   % Mean of each class in Y
end

PhibX = zeros(p,c);
PhibY = zeros(q,c);
for i = 1:c
    PhibX(:,i) = sqrt(nSample(i)) * (classMeanX(:,i)-meanX);
    PhibY(:,i) = sqrt(nSample(i)) * (classMeanY(:,i)-meanY);
end

clear label index cellX cellY meanX meanY classMeanX classMeanY


artSbx = (PhibX') * (PhibX);   % Artificial Sbx (artSbx) is a (c x c) matrix
[eigVecs,eigVals] = eig(artSbx);
eigVals = abs(diag(eigVals));

maxEigVal = max(eigVals);
zeroEigIndx = find(eigVals/maxEigVal<1e-6);
eigVals(zeroEigIndx) = [];
eigVecs(:,zeroEigIndx) = [];

[~,index] = sort(eigVals,'descend');
eigVals = eigVals(index);
eigVecs = eigVecs(:,index);

SbxEigVecs = (PhibX) * (eigVecs);

cx = length(eigVals);   % Rank of Sbx
for i = 1:cx
    SbxEigVecs(:,i) = SbxEigVecs(:,i)/norm(SbxEigVecs(:,i));
end

SbxEigVals = diag(eigVals);                 % SbxEigVals is a (cx x cx) diagonal matrix
Wbx = (SbxEigVecs) * (SbxEigVals^(-1/2));	% Wbx is a (p x cx) matrix which unitizes Sbx

clear index eigVecs eigVals maxEigVal zeroEigIndx
clear PhibX artSbx SbxEigVecs SbxEigVals

artSby = (PhibY') * (PhibY);	% Artificial Sby (artSby) is a (c x c) matrix
[eigVecs,eigVals] = eig(artSby);
eigVals = abs(diag(eigVals));

maxEigVal = max(eigVals);
zeroEigIndx = find(eigVals/maxEigVal<1e-6);
eigVals(zeroEigIndx) = [];
eigVecs(:,zeroEigIndx) = [];

[~,index] = sort(eigVals,'descend');
eigVals = eigVals(index);
eigVecs = eigVecs(:,index);

SbyEigVecs = (PhibY) * (eigVecs);

cy = length(eigVals);      % Rank of Sby
for i = 1:cy
    SbyEigVecs(:,i) = SbyEigVecs(:,i)/norm(SbyEigVecs(:,i));
end

SbyEigVals = diag(eigVals);                  % SbyEigVals is a (cy x cy) diagonal matrix
Wby = (SbyEigVecs) * (SbyEigVals^(-1/2));    % Wby is a (q x cy) matrix which unitizes Sby

clear index eigVecs eigVals maxEigVal zeroEigIndx
clear PhibY artSby SbyEigVecs SbyEigVals


r = min(cx,cy);	% Maximum length of the desired feature vector

Wbx = Wbx(:,1:r);
Wby = Wby(:,1:r);

Xp = Wbx' * X;  % Transform X (pxn) to Xprime (rxn)
Yp = Wby' * Y;  % Transform Y (qxn) to Yprime (rxn)

Sxy = Xp * Yp';	% Between-set covariance matrix

[Wcx,S,Wcy] = svd(Sxy); % Singular Value Decomposition (SVD)

Wcx = Wcx * (S^(-1/2)); % Transformation matrix for Xp
Wcy = Wcy * (S^(-1/2)); % Transformation matrix for Yp

Xs = Wcx' * Xp;	% Transform Xprime to XStar
Ys = Wcy' * Yp;	% Transform Yprime to YStar

Ax = (Wcx') * (Wbx');	% Final transformation Matrix of size (rxp) for X
Ay = (Wcy') * (Wby');	% Final transformation Matrix of size (rxq) for Y
end



fileID = fopen('/content/drive/MyDrive/FGNET/flatten_feature.txt','r');
fileID1 = fopen('/content/drive/MyDrive/FGNET/fc6_feature.txt','r');
fileID2 = fopen('/content/drive/MyDrive/FGNET/fc7_feature.txt','r');
Flatten = [];
FC7 = [];
FC6 = [];
labels = [];
for i=1:1002
    l_str = fgets(fileID);
    Flaten_str = fgets(fileID);
    l_str1 = fgets(fileID1);
    FC6_str = fgets(fileID1);
    l_str2 = fgets(fileID2);
    FC7_str = fgets(fileID2);
    l = str2double(l_str);
    labels = [ labels , l];
    Flatten = [Flatten ;str2double(strsplit(Flaten_str,' '))];
    FC7 = [FC7 ;str2double(strsplit(FC7_str,' '))];
    FC6 = [FC6 ;str2double(strsplit(FC6_str,' '))];
    if mod(i,50)==0
       now()
    end
end
[m,n] = size(Flatten);
[m2,n2] = size(FC6);
[m1,n1] = size(FC7);
FC7=FC7';
FC6=FC6';
Flatten=Flatten';
[Ax1,Ay1,Xs1,Ys1] = dcaFuse(Flatten,FC6,labels);

[Ax2,Ay2,Xs2,Ys2] = dcaFuse(FC7,FC6,labels);

level2_X = Xs1 + Ys1;
level2_Y = Xs2 + Ys2;

[Ax_out,Ay_out,Xs_out,Ys_out] = dcaFuse(level2_X,level2_Y,labels);

final_output = Xs_out + Ys_out;

fid = fopen('/content/fused_feature.csv','w');
t = final_output';
for i=1:1002

   for j=1:81
      fprintf(fid,"%f ",t(i,j));
   end
   fprintf(fid,"\n");
end

fclose(fid);

In [ ]:
f = open('/content/drive/MyDrive/FGNET/features.txt', 'r')
labels = np.zeros((1002))
datas = []
for i in range(1002):
        line = f.readline()
        line = line.replace('\n','')
        line = line.rstrip()
        line = line.split(' ')
        line = np.array(line)
        line = line.astype(np.float64)
        labels[i] = int(line[0])
        datas.append(line[1:82])
datas = np.array(datas)
clf = svm.SVC(kernel='linear', C=2, random_state=64)
scores = cross_val_score(clf, datas, labels, cv=5,)
X_train, X_test, y_train, y_test = train_test_split(datas, labels, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
clf.fit(X_train, y_train)

# Preprocess the testing set features
#X_test = scaler.transform(X_test)
#X_test = scaler.transform(-0.063255 0.057753 0.090585 0.001138 0.007934 0.052132 0.010607 -0.071423 -0.000733 0.001727 0.051481 -0.062546 0.120139 -0.043215 -0.011291 -0.038831 -0.017036 0.080144 0.043525 -0.024920 0.060249 -0.036927 -0.008996 -0.077584 -0.028486 -0.089299 0.073151 -0.165614 0.058591 -0.048085 0.064808 -0.018321 -0.015430 0.039559 0.032330 -0.084407 0.063043 0.009131 0.062507 0.087157 0.002908 0.067805 -0.012294 -0.001077 0.012506 -0.111870 0.016469 -0.059404 0.050445 0.086281 -0.020694 -0.010351 0.023006 -0.068892 0.039647 0.008252 -0.048707 -0.021926 0.067732 -0.011538 0.025676 -0.142955 0.030327 -0.012221 0.033274 -0.125274 -0.002011 -0.068142 -0.012377 0.049638 -0.065519 -0.029196 0.055132 -0.033638 0.026789 -0.044830 -0.025629 0.090513 -0.015603 -0.040551 0.009460 ])
# Evaluate the trained model on the testing set features and labels
y_pred = clf.predict(-0.063255 0.057753 0.090585 0.001138 0.007934 0.052132 0.010607 -0.071423 -0.000733 0.001727 0.051481 -0.062546 0.120139 -0.043215 -0.011291 -0.038831 -0.017036 0.080144 0.043525 -0.024920 0.060249 -0.036927 -0.008996 -0.077584 -0.028486 -0.089299 0.073151 -0.165614 0.058591 -0.048085 0.064808 -0.018321 -0.015430 0.039559 0.032330 -0.084407 0.063043 0.009131 0.062507 0.087157 0.002908 0.067805 -0.012294 -0.001077 0.012506 -0.111870 0.016469 -0.059404 0.050445 0.086281 -0.020694 -0.010351 0.023006 -0.068892 0.039647 0.008252 -0.048707 -0.021926 0.067732 -0.011538 0.025676 -0.142955 0.030327 -0.012221 0.033274 -0.125274 -0.002011 -0.068142 -0.012377 0.049638 -0.065519 -0.029196 0.055132 -0.033638 0.026789 -0.044830 -0.025629 0.090513 -0.015603 -0.040551 0.009460 )
#y_pred = clf.predict([-0.053637,0.027369,0.077081,0.014688,-0.008778,0.029595 -0.006293 0.025410 0.017250 0.054591 0.045669 -0.010900 -0.025981 -0.000872 -0.054870 -0.003126 -0.064221 -0.004755,-0.090718,0.037901,0.060120,0.060956,-0.002453,-0.015528,0.011481,-0.058576,0.013758,-0.039952,-0.013784,-0.052151,0.011175,0.009480,0.027875,0.027457,-0.003757,0.013457,0.035135,-0.139008,0.027137,0.017709,0.087382,-0.065300,0.010226,0.004989,-0.031962,-0.037416,-0.056150,-0.048277,-0.006902,0.063099,-0.022769,-0.006367,0.052272,-0.049616,0.025147,0.034654,-0.040936,-0.064157,0.025855,-0.052775,0.024318,0.004711,0.059159,-0.008051,-0.036318,-0.028619,0.035250,0.009757,0.005044,-0.027601,0.009273,0.010351,0.009913,0.050390,0.022367,-0.016320,-0.002284,0.063261,0.003994,-0.037536,0.045811])
#accuracy = clf.score(X_test, y_test)
#precision, recall, f1_score, _ = metrics.precision_recall_fscore_support(y_test, y_pred, average='weighted')
#scores = cross_val_score(clf, datas, labels, cv=5,)
#print(np.mean(scores))
#print(scores)
#print(accuracy)
#print(precision)
#print(recall)
#print(f1_score)

In [ ]:
print(clf.predict(X_test[3].reshape(1,-1)))

[12.]


In [ ]:
!apt install octave

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aglfn epstool fonts-droid-fallback fonts-freefont-otf fonts-noto-mono
  fonts-urw-base35 ghostscript gnuplot-data gnuplot-qt gsfonts
  imagemagick-6-common info install-info libamd2 libauthen-sasl-perl libcamd2
  libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libdata-dump-perl libemf1
  libencode-locale-perl libfftw3-single3 libfile-listing-perl libfltk-gl1.3
  libfltk1.3 libfont-afm-perl libglpk40 libgraphicsmagick++-q16-12
  libgraphicsmagick-q16-3 libgs9 libgs9-common libhtml-form-perl
  libhtml-format-perl libhtml-parser-perl libhtml-tagset-perl
  libhtml-tree-perl libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libidn11 libijs-0.35
  libio-html-perl libio-socket-ssl-perl libjbig2dec0 liblqr-1-0 liblua5.3-0
  liblwp-mediatypes-perl liblwp-protocol-https-perl libmagick++-6.q16-

In [ ]:
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [969 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,027 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-upd

In [ ]:
!octave -W /content/sample.m

In [ ]:
!octave -W /content/drive/MyDrive/FGNET/main.m

ans =  738979.68499
ans =  738979.68515
ans =  738979.68533
ans =  738979.68550
ans =  738979.68569
ans =  738979.68589
ans =  738979.68607
ans =  738979.68626
ans =  738979.68646
ans =  738979.68667
ans =  738979.68688
ans =  738979.68709
ans =  738979.68732
ans =  738979.68754
ans =  738979.68778
ans =  738979.68802
ans =  738979.68826
ans =  738979.68861
ans =  738979.68887
ans =  738979.68911


In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm,metrics
from sklearn.model_selection import cross_val_score